# System Identification with Archimedes

System identification in Archimedes combines familiar prediction error method (PEM) workflows with automatic differentiation and structured parameter handling. Instead of manually coding gradients or managing flat parameter vectors, you can focus on your model physics while Archimedes handles the optimization details automatically.

This page shows how to implement nonlinear system identification workflows in Python with clean, reliable code that was previously difficult to achieve outside of specialized commercial toolboxes.

## Basic PEM Setup

Let's start with a simple second-order oscillator model. Here's synthetic step response data from a mass-spring-damper system:

```python
import numpy as np
import archimedes as arc

# Load step response data
raw_data = np.loadtxt("oscillator_step.csv", skiprows=1, delimiter="\t")
data = arc.sysid.Timeseries(
    ts=raw_data[:, 0],
    us=raw_data[:, 1].reshape(1, -1),
    ys=raw_data[:, 2].reshape(1, -1),
)
```

Define your system model using standard NumPy operations:

```python
# Continuous-time dynamics: mass-spring-damper
def oscillator_model(t, x, u, p):
    x1, x2 = x
    omega_n, zeta = p
    
    x1_t = x2
    x2_t = -omega_n**2 * x1 - 2*zeta*omega_n*x2 + omega_n**2 * u[0]
    
    return np.hstack([x2, x2_t])

# Observation model
def obs(t, x, u, p):
    return x[0]  # Observe position

# Discretize for system identification
dt = data.ts[1] - data.ts[0]
dyn = arc.discretize(oscillator_model, dt, method="rk4")
```

Now estimate the unknown parameters `[omega_n, zeta]`:

```python
# Set up noise estimates
noise_var = 0.5 * np.var(np.diff(data.ys[0]))
R = noise_var * np.eye(1)  # Measurement noise
Q = 1e-2 * noise_var * np.eye(2)  # Process noise

# Extended Kalman Filter for predictions
ekf = arc.observers.ExtendedKalmanFilter(dyn, obs, Q, R)

# Estimate parameters
params_guess = np.array([2.5, 0.1])  # Initial guess for [omega_n, zeta]
result = arc.sysid.pem(ekf, data, params_guess, x0=np.zeros(2))

print(f"Estimated natural frequency: {result.x[0]:.4f} rad/s")
print(f"Estimated damping ratio:     {result.x[1]:.4f}")
```

That's it! The `pem` function handles gradient computation, optimization, and numerical details automatically.

## Structured Parameters with PyTrees

For complex systems with multiple subsystems, flat parameter vectors become unwieldy. Archimedes supports hierarchical parameter structures using PyTrees:

```python
from archimedes import struct

@struct.pytree_node
class VehicleModel:
    # Drivetrain parameters
    drivetrain: dict  # {'inertia': float, 'damping': float}
    
    # Suspension parameters  
    suspension: dict  # {'stiffness': float, 'damping': float}
    
    def dynamics(self, t, x, u):
        # Access parameters naturally: self.drivetrain['inertia']
        # ... implement vehicle dynamics
        return x_dot

# Create hierarchical parameter guess
params_guess = VehicleModel(
    drivetrain={'inertia': 100.0, 'damping': 5.0},
    suspension={'stiffness': 50000.0, 'damping': 2000.0}
)

# Optimization works directly with the structured model
result = arc.sysid.pem(ekf, data, params_guess, x0=x0_guess)
optimized_model = result.x
```

This approach scales naturally to complex systems while maintaining clear parameter organization.

## Parameter Bounds

Physical systems often have known parameter bounds. Archimedes supports box constraints with the same parameter structure:

```python
# Parameter bounds with same structure as initial guess
lower_bounds = np.array([0.1, 0.01])   # [omega_n_min, zeta_min]
upper_bounds = np.array([10.0, 2.0])   # [omega_n_max, zeta_max]

result = arc.sysid.pem(
    ekf, 
    data, 
    params_guess,
    x0=np.zeros(2),
    bounds=(lower_bounds, upper_bounds)
)
```

For PyTree parameters, create bounds with the same structure:

```python
lower_bounds = VehicleModel(
    drivetrain={'inertia': 10.0, 'damping': 0.1},
    suspension={'stiffness': 1000.0, 'damping': 100.0}
)

upper_bounds = VehicleModel(
    drivetrain={'inertia': 1000.0, 'damping': 50.0},
    suspension={'stiffness': 100000.0, 'damping': 10000.0}
)

result = arc.sysid.pem(ekf, data, params_guess, bounds=(lower_bounds, upper_bounds))
```

## Multiple Experiments

System identification often uses multiple datasets to ensure model robustness. Pass a list of datasets to optimize against all experiments simultaneously:

```python
# Load different experiment types
step_data = arc.sysid.Timeseries(...)    # Step response
chirp_data = arc.sysid.Timeseries(...)   # Frequency sweep  
ramp_data = arc.sysid.Timeseries(...)    # Ramp input

# Optimize against all datasets
result = arc.sysid.pem(
    ekf,
    [step_data, chirp_data, ramp_data],
    params_guess,
    x0=[np.zeros(2), None, np.zeros(2)]  # Known/unknown initial conditions
)
```

When `x0` contains `None`, the initial condition for that experiment is estimated along with the parameters.

## Validation and Next Steps

Always validate your identified model against independent test data:

```python
# Simulate identified model
def ode_rhs(t, x, params):
    u = np.interp(t, test_data.ts, test_data.us[0])
    return oscillator_model(t, x, u, params)

xs_pred = arc.odeint(ode_rhs, t_span, x0_test, args=(result.x,), t_eval=test_data.ts)

# Compare predictions to test measurements
plt.plot(test_data.ts, test_data.ys[0], label='Test Data')
plt.plot(test_data.ts, xs_pred[0], label='Model Prediction')
```

## Advanced Usage

For custom objectives or nonlinear constraints, use the lower-level `PEMObjective` interface:

```python
# Create PEM objective manually
pem_obj = arc.sysid.PEMObjective(ekf, data, P0=np.eye(2)*1e-4, x0=x0_guess)

# Custom optimization with additional constraints
def custom_objective(params):
    return pem_obj(params) + custom_penalty(params)

result = arc.optimize.minimize(custom_objective, params_guess, constr=constraints)
```

This enables advanced workflows like enforcing steady-state behavior, energy conservation, or other physical constraints during parameter estimation.